# TPSC and the Mermin-Wagner theorem

The Mermin-Wagner theorem states that a continuous-symmetry cannot be broken at finite temperature in two dimensions. So, antiferromagnetism is impossible in two dimensions, contrary to the prediction of RPA. Here is a heuristic proof. Assume that the spins are collinear in the $z$ direction. Then the free energy density contains a term proportional to $(\nabla S^z)^2/2$. In Fourier space, this becomes $-q^2|S^z(\mathbf{q})|^2/2$. In the long wavelength limit, where fluctuations are slow, we can use the classical equipartition theorem so that 

$$|S^z(\mathbf{q})|^2=\frac{k_BT}{q^2}$$. 

This means that the local moment diverges because of an infrared divergence

$$\left<(S^z)^2\right>\sim \int d^2q |S^z(\mathbf{q})|^2\sim \int d^2q\frac{k_BT}{q^2}=\infty$$. 

We come to an absurdity, which proves that the hypothesis is wrong, there is no long-range order. 

To see that TPSC satisfies the Mermin-Wagner theorem, first note that the spin susceptibility has the following spectrral representation

$$\chi_{sp}(\mathbf{q},i\omega_n)=\int\frac{d\omega}{\pi}\frac{\chi_{sp}''(\mathbf{q},\omega)}{\omega-i\omega_n}=\int\frac{d\omega}{\pi}\frac{\chi_{sp}''(\mathbf{q},\omega)\omega}{\omega^2+(\omega_n)^2}.$$

The last equality follows from the fact $\chi_{sp}''(\mathbf{q},\omega)$ is odd in frequency. This last result shows that the finite Matsubara frequencies should be regular. The largest contribution is the zero-Matsubara frequency one. This allows us to give a rough idea of why the theorem is satisfied by focusing on the zero Matsubara frequency contribution. 

Let us then write the self-consistency condition for $U_{sp}$ as follows :

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q}} \frac{\chi_0(\mathbf{q},0)}{1-\frac{U_{sp}}{2}\chi_0(\mathbf{q},0)}=n-2\left< n_\uparrow n_\downarrow\right>-C(T)
\end{equation}

where $C(T)$ contains the non-singular contribution of the finite Matsubara frequencies. 

Calling the right-hand side $C'(T)$, expanding the denominator around the maximum at $\mathbf{Q}=(\pi,\pi)$ and shifting the origin of the wave vector integration to $\mathbf{Q}=(\pi,\pi)$, the self-consistency condition can be written as  

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q}} \frac{A}{\xi^{-2}+q^2}=C'(T)
\end{equation}
where $A$ is a constant and $\xi$ the correlation length contains the value of $U_{sp}$. Since the right-hand side is finite, $\xi$ adjusts itself not to become infinite, otherwise the left-hand side diverges. The divergence of the susceptibility can occur only at $T=0$ where we cannot treat the non-zero Matsubara frequencies separately. 

Here we compute a temperature sweep for $U=4$

Note that the spin structure factor, that we will compute, is an equal-time correlation function. It is given by 
(\mathbf{q},\omega)
$$S(\mathbf{q})\equiv T\sum_n \chi_{sp}(\mathbf{q},i\omega_n)$$.

In [ ]:
from pytriqs.plot.mpl_interface import plt
import numpy as np
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, Idx, inverse
from pytriqs.archive import HDFArchive
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,9) # set default size for all figuresfrom pytriqs.archive import HDFArchivefrom pytriqs.gf import Gf, inverse

In [ ]:
%reload_ext cpp2py.magic

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (k,omega) -> Complex number
using g_k_w_type = gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued>;
using g_r_t_type = gf<cartesian_product<cyclic_lattice, imtime>, scalar_valued>;

g_k_w_type bubble(g_k_w_type g0) {
    
    // Fourier Transformation of k, \omega to obtain g(r,t)
    auto grt = make_gf_from_fourier<0,1>(g0);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mr, mt] = grt.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (r, tau) with this new mesh.
    auto chi0 = g_r_t_type{{mr, mtb}};

    // we fill chi : chi(tau, r) = g(beta - tau, -r) * g(tau, r)
    for (auto const &r : mr)      
        for (auto const &t : mtb) 
            chi0[r, t] = grt(-r, beta - t) * grt(r, t); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

## Fixed parameters

In [ ]:
U = 4.0
t = 1.0
n_k = 32 # Number of k points per direction
n_w = 128 # Number of Matsubara frequencies

# Two unit vectors in R3
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)])
BZ = BrillouinZone(BL)
kmesh = MeshBrillouinZone(BZ, n_k = n_k)

## Functions (were obtained in previous notebooks, please just run)

In [ ]:
def eps(kx,ky):
    return -2 * t* (np.cos(kx) + np.cos(ky))

def get_chi0(beta):
    
    # Init g0
    wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)
    w = np.tensordot(np.ones(n_k*n_k), list(wmesh.values()), 0)
    k = np.tensordot(list(kmesh.values()), np.ones(2*n_w), 0)
    kx, ky = k[:,0,:], k[:,1,:]

    g0 = Gf(mesh = MeshProduct(kmesh, wmesh), target_shape = [])
    g0.data[:,:] = 1 / (w - eps(kx,ky))
    
    return bubble(g0)

def chi_rpa(chi0_wk, U):
    chi_rpa = chi0_kw.copy()
    chi_rpa = chi0_kw * inverse(1 - U * chi0_kw)
    return chi_rpa

# Should we have some kind of trace in TRIQS?
def trace_chi_kw(chi_kw):
    kmesh, wmesh = chi_kw.mesh.components
    # tail correction FIXME!!!
    # We neglect the tail. It is justified at those temperatures
    trace = chi_kw.data.sum() / len(kmesh) / wmesh.beta
    assert(np.abs(trace.imag) < 1e-10)
    return trace.real

def Usp_root_problem(Usp, chi0, n, U):
    tr_chi_sp = trace_chi_kw(chi_rpa(chi0, U=Usp[0]))
    diff = 2*tr_chi_sp + 0.5 * Usp/U * n**2 - n
    return diff

def Uch_root_problem(Uch, chi0, n, U, docc):
    tr_chi_ch = trace_chi_kw(chi_rpa(chi0, U=-Uch[0]))
    diff = 2*tr_chi_ch - 2 * docc - n + n**2
    return diff

def solve_Usp_and_Uch(chi0, U, n, Usp0=0.1, Uch0=0.1):
    Usp = fsolve(Usp_root_problem, Usp0, args=(chi0, n, U), xtol=1e-2)[0]
    docc = 0.25 * Usp / U * n**2
    Uch = fsolve(Uch_root_problem, Uch0, args=(chi0, n, U, docc), xtol=1e-2)[0]
    return Usp, Uch, docc

## RPA susceptibility as a function of temprature (susceptibility from previous notebooks, please just run)

In [ ]:
T_rpa_vec = np.concatenate((np.arange(4., 3., -1.), np.arange(3, 0.75, -0.2)))
S_rpa_vec = np.zeros_like(T_rpa_vec)

print ''.join('| %-11s' % s for s in ['T', 'beta', 'S_rpa']), '|'
print '-'*41

for idx, T in enumerate(T_rpa_vec):

    beta = 1. / T
    chi0_kw = get_chi0(beta)
    chi = chi_rpa(chi0_kw, U)
    
    S_rpa = chi[Idx(n_k/2, n_k/2, 0), :].data.sum().real # FIXME: Replace with .density() call
    S_rpa_vec[idx] = S_rpa
    
    print ''.join('| %4.4E ' % x for x in [T, beta, S_rpa]), '|'

## TPSC structure factor as a function of temperature

We want to reproduce the following figure from the
__[paper](https://jp1.journaldephysique.org/articles/jp1/abs/1997/11/jp1v7p1309/jp1v7p1309.html)__ :
<img src="./img/Fig7.png" alt="Drawing" style="width: 250px;"/>

by calling the function tpsc for a grid of values of the bare U.

This time we fix $U=4$ and vary temperature. Using the ansatz $U_{sp}\left<n_\uparrow\right> \left<n_\downarrow\right>=U\left<n_\uparrow n_\downarrow\right>$, the spin susceptibility obeys

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \frac{\chi_0(\mathbf{q},iq_n)}{1-\frac{U\left<n_\uparrow n_\downarrow\right>}{2\left<n_\uparrow\right> \left<n_\downarrow\right>}\chi_0(\mathbf{q},iq_n)}=n-2\left< n_\uparrow n_\downarrow\right>
\end{equation}

When the susceptibility increases, $\left<n_\uparrow n_\downarrow\right>$ on the right-hand side decreases, but then the denominator of the spin susceptibility will lead to a decreas in susceptibility.

More rigorously, we can see that dimension is important here. Let us repeat the argument at the beginning of the notebook. The right-hand side of the equation cannot diverge. Also, on the left-hand side, note that the most divergent contribution is the zero Matsubara frequency, as one can see from the spectral representation and $\chi''(\mathbf{q},\omega)=-\chi''(\mathbf{q},-\omega)$

\begin{equation}
\chi(\mathbf{q},iq_n)=\int \frac{d\omega}{\pi}\frac{\chi''(\mathbf{q},\omega)}{\omega-iq_n}=\int \frac{d\omega}{\pi}\frac{\omega\chi''(\mathbf{q},\omega)}{\omega^2+q_n^2}.
\end{equation}

Using these results, the non-singular finite Matsubara frequency terms can be put on the right-hand side of the sum rule and all that is left is 

\begin{equation}
T\int d^2q \frac{a}{\xi^{2}+q^2}\sim C'(T)
\end{equation}

where we have expanded the susceptibility around $(\pi,\pi)$, gone from sum to integral and shifted the origin of integration so that now $\mathbf{q}$ is the deviation from $(\pi,\pi)$. On dimensional grounds, the left-hand side is logarithmic in two dimensions so that the correlation length scales like $\exp(C'(T)/T)$.


In [ ]:
n = 1.
Usp, Uch = 1., 1. # initial guess

T_tpsc_vec = np.array([ 4., 3., 
                       2.5, 2.0, 1.5, 1.2, 1.0, 
                       0.8, 0.6, 0.4, 0.35, 0.3, 0.25])
S_tpsc_vec = np.zeros_like(T_tpsc_vec)
U_sp_vec = np.zeros_like(T_tpsc_vec)

print ''.join('| %-11s' % s for s in ['T', 'beta', 'Usp', 'Uch', 'docc', 'S_tpsc']), '|'
print '-'*80

for idx, T in enumerate(T_tpsc_vec):

    beta = 1. / T    
    chi0_kw = get_chi0(beta) 
    
    Usp, Uch, docc = solve_Usp_and_Uch(chi0_kw, U, n, Usp0=Usp, Uch0=Uch)
    
    chi_sp_wk = chi_rpa(chi0_kw, Usp)
    S_tpsc = chi_sp_wk[Idx(n_k/2, n_k/2, 0),:].data.sum().real

    S_tpsc_vec[idx], U_sp_vec[idx] = S_tpsc, Usp

    print ''.join('| %4.4E ' % x for x in [T, beta, Usp, Uch, docc, S_tpsc]), '|'


## Plots results

In [ ]:
# This is to compare with the paper and RPA
plt.subplot(221)
plt.title(r'$U = %2.2f$' % U)
plt.plot(T_rpa_vec, S_rpa_vec, 'o-', label=r'$S_{RPA}$', alpha=0.5)
plt.plot(T_tpsc_vec, S_tpsc_vec, 'o-', label=r'$S_{TPSC}$', alpha=0.5)
plt.legend(loc='best')
plt.xlabel(r'$T$')

# By plotting the inverse of the structure factor, we look for a phase transition. 
plt.subplot(222)
plt.plot(T_rpa_vec, 1./S_rpa_vec, 'o-', alpha=0.5, label=r'$S_{RPA}^{-1}$')
plt.plot(T_tpsc_vec, 1./S_tpsc_vec, 'o-', alpha=0.5, label=r'$S_{TPSC}^{-1}$')
plt.legend(loc='best')
plt.xlabel(r'$T$'); plt.grid()

# This is a blow up of the low temperature result. 
plt.subplot(223)
plt.plot(T_rpa_vec, 1./S_rpa_vec, 'o-', alpha=0.5, label=r'$S_{RPA}^{-1}$')
plt.plot(T_tpsc_vec, 1./S_tpsc_vec, 'o-', alpha=0.5, label=r'$S_{TPSC}^{-1}$')
plt.legend(loc='best'); plt.xlim([0, 2]); plt.ylim([-0.1, 2.5])
plt.xlabel(r'$T$'); plt.grid()

# This shows the behavior of U_sp as a function of temperature.
plt.subplot(224)
plt.plot(T_rpa_vec, U + 0*T_rpa_vec, 'o-', alpha=0.5, label=r'$U$')
plt.plot(T_tpsc_vec, U_sp_vec, 'o-', alpha=0.5, label=r'$U_{sp}$')
plt.legend(loc='best')
plt.xlabel(r'$T$')

plt.tight_layout()
plt.show()

## Question
* There seems to be a finite transition temperature. If there is a numerical problem leading to that, can you identify it?

## Answer 

* The zero-Matsubara frequency contribution to the local moment sum-rule is a logarithmically divergent integral when the correlation length is infinite. Since the right-hand side of the sum-rule is finite, the correlation length is forced to remain finite. If the momentum mesh is discrete and does not include $0$, the discrete numerical integral converges even for an infinite correlation length, so there is a finite transition temperature. See the remarks below for how to solve this problem. It amounts to basically doing a careful analysis of the asymptotic behavior, subtracting this asymptotic behavior in the numerical integral and adding the analytical expression for the asymptotic part.   

## Remarks
* To evaluate the integrals and sums entering the sum rules accurately, one must use several tricks that are described in appendix B of this [paper](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.53.14236).
* The asymptotic behavior of the self-consistency equation is done in a more rigorous way as an intermediate step in this [discussion of the critical behavior](http://iopscience.iop.org/article/10.1209/0295-5075/84/37013/meta).
* $U_{sp}$ vanishing as $T->0$ is unphysical. It is a consequence of the fact that to avoid the phase transition, $U_{sp}$ must be smaller than the mean-field value $$ U_c^{(RPA)} = \frac{2}{\chi_0(0, \mathbf{Q}_{AF})} $$ which is zero at $T=0$ because the susceptibility diverges there for the perfectly nested Fermi surface.
* The results can be trusted at temperatures not too far below the rapid crossover in the renormalized classical regime where $\xi$ starts to grow exponentially. In particular, the sudden fall of $U_{sp}$ around $T=0.5$ is seen by other methods in this [comment](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.90.099702).
* The internal consistency condition $Tr[\Sigma G]=2U\left< n_\uparrow n_\downarrow \right>$ can be used to estimate the domain of validity of the approach.                                                                  